In [1]:
import bittensor
import psutil

In [4]:
proc = psutil.Process()
print(proc.open_files())

[popenfile(path='/home/athrunxyz/.ipython/profile_default/history.sqlite', fd=48, position=0, mode='r+', flags=688130), popenfile(path='/home/athrunxyz/.ipython/profile_default/history.sqlite', fd=49, position=0, mode='r+', flags=688130), popenfile(path='/home/athrunxyz/.ipython/profile_default/history.sqlite-journal', fd=57, position=0, mode='r+', flags=688130)]


In [2]:
wallet = bittensor.wallet()

In [3]:
wallet.hotkey

<Keypair (ss58_address=5FC6iTwsR3oJs4db92uBTjPBs6HuA1H4bPCs6aGbvdLxa88m)>

In [10]:
from tqdm import tqdm
for i in tqdm(range(100)):
    print(i)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 4771.89it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
